<a href="https://colab.research.google.com/github/junaid537/pytorch/blob/main/%E2%80%9CHW2_Junaid_Khalidi_Py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch